# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [36]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
# contar a quantidade de missing
soma_missing = sinasc.isnull().sum()
soma_missing


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
# verificando a quantidade de valores nulos nas colunas selecionadas
soma_missing_selecionados = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()
soma_missing_selecionados

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
# 4) seu código aqui
# removendo valores nulos da coluna APGAR5, visto que é nossa variável principal
sinasc_selecionado = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_selecionado_apgar5ok = sinasc_selecionado.dropna(subset=['APGAR5'])
sinasc_selecionado_apgar5ok.head(3)



,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0


In [5]:
# verificando se há valores nulos na coluna APGAR5
print(sinasc_selecionado_apgar5ok[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum())

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [9]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. 
# Vamos assumir que o não preenchido é o mesmo que o código 9.



sinasc_selecionado_apgar5ok.loc[sinasc_selecionado_apgar5ok['ESTCIVMAE'].isna(), 'ESTCIVMAE'] = 9
sinasc_selecionado_apgar5ok.loc[sinasc_selecionado_apgar5ok['CONSULTAS'].isna(), 'CONSULTAS'] = 9

sinasc_selecionado_apgar5ok.head(2)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0


In [10]:
sinasc_selecionado_apgar5ok[['ESTCIVMAE','CONSULTAS']].isnull().sum()

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [23]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.

df = sinasc_selecionado_apgar5ok.copy()

df['QTDFILVIVO'] = df['QTDFILVIVO'].fillna(0)
df[['QTDFILVIVO']].isnull().sum()


QTDFILVIVO    0
dtype: int64

In [27]:
# 7) seu código aqui
df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26925 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     26925 non-null  int64  
 1   IDADEMAE    26925 non-null  int64  
 2   ESTCIVMAE   26925 non-null  float64
 3   ESCMAE      26615 non-null  object 
 4   QTDFILVIVO  26925 non-null  float64
 5   GESTACAO    25709 non-null  object 
 6   GRAVIDEZ    26849 non-null  object 
 7   CONSULTAS   26925 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 3.1+ MB


In [32]:
df.describe()

,LOCNASC,IDADEMAE,ESTCIVMAE,QTDFILVIVO,CONSULTAS,APGAR5
count,26925.000000,26925.000000,26925.000000,26925.000000,26925.000000,26925.000000
mean,1.007168,26.095339,2.620241,0.989601,3.578793,9.179164
std,0.126025,6.386409,1.799713,1.158913,0.785540,0.790574
min,1.000000,11.000000,1.000000,0.000000,1.000000,0.000000
25%,1.000000,21.000000,1.000000,0.000000,3.000000,9.000000
50%,1.000000,26.000000,2.000000,1.000000,4.000000,9.000000
75%,1.000000,31.000000,5.000000,1.000000,4.000000,10.000000
max,4.000000,53.000000,9.000000,30.000000,9.000000,10.000000


In [33]:
# 8) seu código aqui
# Vamos assumir que o não preenchido é o mesmo que o código 9.


df.loc[df['ESCMAE'].isna(), 'ESCMAE'] = 9

df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [34]:
df.loc[df['GESTACAO'].isna(), 'GESTACAO'] = 9

df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()

LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ      76
CONSULTAS      0
APGAR5         0
dtype: int64

In [35]:
df.loc[df['GRAVIDEZ'].isna(), 'GRAVIDEZ'] = 9

df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [37]:
# 8) Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
def categoria_apgar5(apgar_score):
    if 0 <= apgar_score <= 3:
        return 'asfixia grave'
    elif 4 <= apgar_score <= 5:
        return 'asfixia moderada'
    elif 6 <= apgar_score <= 7:
        return 'asfixia leve'
    elif 8 <= apgar_score <= 10:
        return 'normal'
    else:
        return 'valor inválido'
    
# aplicando a função na coluna Categoria    
df['Categoria'] = df['APGAR5'].apply(categoria_apgar5) 

# verificando a frequência de cada classificação
df['Categoria'].value_counts()



normal              26463
asfixia leve          320
asfixia grave          74
asfixia moderada       68
Name: Categoria, dtype: int64

In [43]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
# Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

# Suponha que 'df' seja o seu DataFrame
df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26925 entries, 0 to 27027
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   locnasc     26925 non-null  int64  
 1   idademae    26925 non-null  int64  
 2   estcivmae   26925 non-null  float64
 3   escmae      26925 non-null  object 
 4   qtdfilvivo  26925 non-null  float64
 5   gestacao    26925 non-null  object 
 6   gravidez    26925 non-null  object 
 7   consultas   26925 non-null  int64  
 8   apgar5      26925 non-null  float64
 9   categoria   26925 non-null  object 
dtypes: float64(3), int64(3), object(4)
memory usage: 3.3+ MB
